## Import

In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [ ]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

## Model Define, Train and Inference

In [22]:
# Read in price data
train = pd.read_csv("./train.csv", parse_dates=True, index_col="일자")
train = train.reset_index()
train.columns = ['date', 'ticker', 'firm', 'volume', 'open', 'high', 'low', 'close']
train.set_index('date', inplace=True)
train

,ticker,firm,volume,open,high,low,close
date,,,,,,,
2021-06-01,A060310,3S,166690,2890,2970,2885,2920
2021-06-01,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2021-06-01,A006840,AK홀딩스,103691,35500,35600,34150,34400
2021-06-01,A054620,APS,462544,14600,14950,13800,14950
2021-06-01,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...
2023-05-30,A189980,흥국에프엔비,272284,3005,3035,2955,2980
2023-05-30,A000540,흥국화재,50218,3250,3255,3195,3215
2023-05-30,A003280,흥아해운,130664,1344,1395,1340,1370


In [4]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['ticker', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['ticker'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    
    # 학습 데이터 생성
    train_close = train[train['ticker'] == code][['date', 'close']]
    train_close['date'] = pd.to_datetime(train_close['date'], format='%Y%m%d')
    train_close.set_index('date', inplace=True)
    tc = train_close['close']
    
    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측
    
    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    
    # 결과 저장
    results_df = results_df.append({'ticker': code, 'final_return': final_return}, ignore_index=True)

100%|██████████| 2000/2000 [08:48<00:00,  3.78it/s]


In [5]:
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df

,종목코드,final_return,순위
0,A060310,-0.056272,1992
1,A095570,-0.002980,1690
2,A006840,0.003040,350
3,A054620,0.001867,485
4,A265520,0.002090,449
...,...,...,...
1995,A189980,0.000630,731
1996,A000540,0.002513,392
1997,A003280,0.001430,552
1998,A037440,-0.000038,1085


## Submit

In [6]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [8]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,497
1,A000040,1652
2,A000050,869
3,A000070,1080
4,A000080,1204
...,...,...
1995,A375500,753
1996,A378850,558
1997,A383220,616
1998,A383310,242


In [9]:
baseline_submission.to_csv('baseline_submission.csv', index=False)